In [2]:
from ipynb.fs.full.capacity_scaling import Network, capacity_scaling

Number of flow updates: 7
Number of flow updates: 7


In [3]:
def parse(filename) -> Network:
    """
    Parses a network file following the DIMACS problem specification 
    structure and transforms it into a Network object
    
    Some elements of the specification:
    - Lines starting in c are comments
    - Lines starting in p explain what problem to solve (can be ignored, 
      we only consider minimum-cost flow problems)
    - Lines starting in n define nodes
    - Lines starting in a define arcs (edges)
    
    Args:
        filename: name of the file containing the network data
        
    Returns:
        The corresponding Network object
    """
    # Lines we can ignore
    ignore_list = ['c', 'p']
    
    file = open(filename, 'r')
    
    # Nodes is a hashmap from node values to their supply
    nodes = {}
    # Edges is a hashmap from edges to a tuple with their capacity and cost
    edges = {}
    
    for line in file:
        if len(line) > 0 and line[0] not in ignore_list:
            if line[0] == 'n':
                # Node parsing
                node = [int(elem) for elem in line.split(' ')[1:]]
                nodes[node[0]] = node[1]
            elif line[0] == 'a':
                arc = [int(elem) for elem in line.split(' ')[1:]]
                node1 = arc[0]
                node2 = arc[1]
                capacity = arc[3]
                cost = arc[4]
                
                # Only nodes with non-zero supply are in a "node line"
                if node1 not in nodes:
                    nodes[node1] = 0
                if node2 not in nodes:
                    nodes[node2] = 0
                if (node1, node2) in edges:
                    # TODO not amazing (reaverages every time)
                    old_capacity, old_cost = edges[(node1, node2)]
                    new_cost = old_cost * old_capacity + cost * capacity
                    new_cost /= (old_capacity + capacity)
                    edges[(node1, node2)] = (old_capacity + capacity, new_cost)
                else:
                    edges[(node1, node2)] = (capacity, cost)
    file.close()
    
    capacities, costs = zip(*edges.values())
    network = Network(nodes.keys(), edges.keys(), capacities, costs, nodes.values()) 
    
    print(f"This dataset contains: {len(nodes.keys())} nodes and {len(edges.keys())} edges")

    return network

In [4]:
network = parse("resources/road_flow_01_DC_a.txt")

This dataset contains: 9559 nodes and 29682 edges


In [5]:
p = {v: 0 for v in network.V}
f, p = capacity_scaling(network, p)

Number of flow updates: 595


In [6]:
f, p = capacity_scaling(network, p)

UnboundLocalError: local variable 'out_path' referenced before assignment